In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.metrics import accuracy_score, classification_report,f1_score, confusion_matrix, roc_auc_score, matthews_corrcoef

## twenty percentage

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/train_20.0_mda_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/test_20.0_mda_selected.csv")

In [5]:
train.head(2)

,f_25,f_45,f_52,f_104,f_125,f_197,f_205,f_218,f_222,f_241,...,f_171,f_172,f_173,f_174,f_175,f_176,f_177,f_178,f_179,label
0,0.883563,0.0,0.0,0.000000,0.0,0.0,0.334627,2.715299,0.000000,0.0,...,0.0,0.28420,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6.0
1,0.000000,0.0,0.0,4.678092,0.0,0.0,0.000000,0.000000,0.663257,0.0,...,0.0,4.29303,0.0,0.0,0.0,0.0,0.0,0.0,2.015075,3.0


In [6]:
x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 204)
test features shape: (700, 204)


In [8]:
# knn
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors=30,
    weights='distance',
    algorithm='kd_tree',
    leaf_size=50
)
knn.fit(x_train, y_train)

# n_neighbors: 80

print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9128571428571428


In [9]:
# svm
from sklearn.svm import SVC

svm = SVC(
    kernel='rbf',
    C=1.5,
    gamma='scale',
    probability=True,
    decision_function_shape='ovr',
    coef0=0.0,
    tol=0.0001
)
svm.fit(x_train, y_train)

print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9428571428571428
test accuracy of svm 0.9157142857142857


In [10]:
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9428571428571428
test accuracy of svm 0.9157142857142857


In [11]:
# rf
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=300, criterion='entropy',
    min_samples_split=4,
    max_features='log2',
    ccp_alpha=0.0
)
rfc.fit(x_train, y_train)

print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  0.9967857142857143
test accuracy of rfc 0.9157142857142857


In [12]:
# mlp
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    activation='relu',
    solver='sgd', alpha=0.0001, batch_size='auto', learning_rate='constant',
    learning_rate_init=0.001, power_t=0.5, max_iter=500, momentum=0.9,
    beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10,
    max_fun=15000
)
mlp.fit(x_train, y_train)

print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of mlp:  0.9464285714285714
test accuracy of mlp 0.9242857142857143


In [13]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

In [14]:
probs_model1_val.shape

(2800, 8)

In [15]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [16]:
meta_features_train[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       8.30500132e-03, 2.63822472e-03, 4.86245438e-03, 6.58046336e-03,
       1.16006024e-02, 5.12527470e-03, 9.58828826e-01, 2.05915274e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       9.30614087e-10, 3.64838762e-10, 1.12182027e-10, 3.73922422e-10,
       1.12098384e-08, 4.19709654e-10, 9.99999976e-01, 1.09917219e-08])

In [64]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.8, C=0.5, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='newton-cholesky', max_iter=600, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

meta_model.fit(meta_features_train, y_train)

# liblinear', 'newton-cholesky', 'lbfgs', 'newton-cg', 'saga', 'sag'

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9142857142857143


In [65]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9142857142857143
MCC : 0.9014109425467218
F1 : 0.9152335486854672
AUC: 0.9929048342259901
Sensitivity: 0.9220831097108899
Specivity: 0.9736230398682074


## forty percent

In [66]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/train_40.0_mda_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/test_40.0_mda_selected.csv")

train.head(2)

,f_25,f_45,f_52,f_104,f_125,f_197,f_205,f_218,f_222,f_241,...,f_394,f_395,f_396,f_397,f_398,f_399,f_400,f_401,f_402,label
0,0.883563,0.0,0.0,0.000000,0.0,0.0,0.334627,2.715299,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1,0.000000,0.0,0.0,4.678092,0.0,0.0,0.000000,0.000000,0.663257,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [67]:
x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 409)
test features shape: (700, 409)


In [ ]:
# # knn
# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier(
#     n_neighbors=9,
#     weights='distance',
#     algorithm='auto',
#     leaf_size=50
# )
# knn.fit(x_train, y_train)

KNeighborsClassifier(leaf_size=50, n_neighbors=9, weights='distance')

In [68]:
knn.fit(x_train, y_train)
print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9171428571428571


In [ ]:
# # svm
# from sklearn.svm import SVC

# svm = SVC(
#     kernel='rbf',
#     C=1.5,
#     gamma='scale',
#     probability=True,
#     decision_function_shape='ovr'
# )
# svm.fit(x_train, y_train)

SVC(C=1.5, probability=True)

In [69]:
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9421428571428572
test accuracy of svm 0.9185714285714286


In [ ]:
# # rf
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(
#     n_estimators=1200,
#     max_depth=20,
#     min_samples_leaf=1,
#     random_state=42
# )
# rfc.fit(x_train, y_train)

In [70]:
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  0.9967857142857143
test accuracy of rfc 0.9157142857142857


In [75]:
# mlp
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    activation='relu',
    solver='adam', alpha=0.001, batch_size='auto', learning_rate='constant',
    learning_rate_init=0.15, power_t=0.5, max_iter=700, momentum=0.9,
    beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10,
    max_fun=15000
)
mlp.fit(x_train, y_train)

MLPClassifier(alpha=0.001, learning_rate_init=0.15, max_iter=700)

In [76]:
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of mlp:  0.9528571428571428
test accuracy of mlp 0.8857142857142857


In [77]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)


In [78]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [80]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.9, C=0.7, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='sag', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

meta_model.fit(meta_features_train, y_train)


print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  0.9996428571428572
test accuracy of meta_model 0.9185714285714286


In [81]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9185714285714286
MCC : 0.9066563443185149
F1 : 0.9197688212464518
AUC: 0.9904224190611116
Sensitivity: 0.9260812450710463
Specivity: 0.9734291125632428


In [ ]:

# save model
file_name = '/content/drive/MyDrive/Filters_colorectal/RFE/fs_best_40.pkl'
with open(file_name, 'wb') as model_file:
    pickle.dump(meta_model, model_file)

## fifty percentage

In [82]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/train_50.0_mda_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/test_50.0_mda_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 512)
test features shape: (700, 512)


In [85]:
# knn = KNeighborsClassifier(
#     n_neighbors=80,
#     weights='distance',
#     algorithm='kd_tree',
#     leaf_size=50
# )
# knn.fit(x_train, y_train)

# n_neighbors: 80

knn = KNeighborsClassifier(
    n_neighbors=50,
    weights='uniform',
    algorithm='brute',
    leaf_size=50
)
knn.fit(x_train, y_train)

print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  0.9364285714285714
test accuracy of knn 0.9142857142857143


In [87]:
# svm = SVC(
#     kernel='rbf',
#     C=1.5,
#     gamma='scale',
#     probability=True,
#     decision_function_shape='ovr',
#     coef0=0.0,
#     tol=0.0001
# )

svm = SVC(
    kernel='linear',
    C=1.5,
    gamma='scale',
    probability=True,
    decision_function_shape='ovr',
    coef0=0.0,
    tol=0.0001
)

svm.fit(x_train, y_train)

print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9785714285714285
test accuracy of svm 0.9085714285714286


In [90]:
# rf
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=200, criterion='gini',
    min_samples_split=4,
    max_features='sqrt',
    ccp_alpha=0.01
)
rfc.fit(x_train, y_train)

print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  0.9382142857142857
test accuracy of rfc 0.9042857142857142


In [91]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  0.9364285714285714
test accuracy of knn 0.9142857142857143
training accuracy of svm:  0.9785714285714285
test accuracy of svm 0.9085714285714286
training accuracy of rfc:  0.9392857142857143
test accuracy of rfc 0.9028571428571428
training accuracy of mlp:  0.9446428571428571
test accuracy of mlp 0.9028571428571428


In [92]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)


In [93]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [107]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.4, C=1.5, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='newton-cg', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg


meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  0.9778571428571429
test accuracy of meta_model 0.9071428571428571


In [108]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9071428571428571
MCC : 0.8934087424214293
F1 : 0.9089515274111898
AUC: 0.990339603697979
Sensitivity: 0.9155275805224704
Specivity: 0.9693915526128689


## sixty percent

In [109]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/train_60.0_mda_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/test_60.0_mda_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 614)
test features shape: (700, 614)


In [110]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  0.9382142857142857
test accuracy of knn 0.9157142857142857
training accuracy of svm:  0.9803571428571428
test accuracy of svm 0.8957142857142857
training accuracy of rfc:  0.9378571428571428
test accuracy of rfc 0.9071428571428571
training accuracy of mlp:  0.9532142857142857
test accuracy of mlp 0.8928571428571429


In [111]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [113]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=0.9, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='liblinear', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg
# tol=0.5, C=0.05 -> l1, saga

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  0.9810714285714286
test accuracy of meta_model 0.9014285714285715


In [114]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9014285714285715
MCC : 0.8868514689873382
F1 : 0.9032529173086317
AUC: 0.9897961078425785
Sensitivity: 0.9104098337480553
Specivity: 0.9692362048732476


## eighty percentage

In [115]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/train_80.0_mda_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/MDA/test_80.0_mda_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 819)
test features shape: (700, 819)


In [116]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  0.9353571428571429
test accuracy of knn 0.9228571428571428
training accuracy of svm:  0.9839285714285714
test accuracy of svm 0.9057142857142857
training accuracy of rfc:  0.9392857142857143
test accuracy of rfc 0.9028571428571428
training accuracy of mlp:  0.9482142857142857
test accuracy of mlp 0.9


In [117]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [125]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.01, C=0.1, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='saga', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg
# tol=0.5, C=0.05 -> l1, saga

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  0.9817857142857143
test accuracy of meta_model 0.9042857142857142


In [126]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9042857142857142
MCC : 0.8901998919779334
F1 : 0.9061374016911578
AUC: 0.9860443418957856
Sensitivity: 0.9132997106290401
Specivity: 0.9692362048732476
